In [4]:
# Queueing for a constant stream of images to do sequential tasks: download, resize, upload and etc.
from threading import Thread, Lock
from collections import deque
from random import randint
from time import sleep

def download(item):
    print('download..')
    sleep(randint(1,3))
    return item

def resize(item):
    print('resize..')
    sleep(randint(1,3))
    return item 

def upload(item):
    print('upload...')
    sleep(randint(1,3))
    return item 

class MyQueue(object):
    def __init__(self):
        self.items = deque()
        self.lock = Lock()
        
    def put(self, item):
        with self.lock:
            self.items.append(item)
            
    def get(self):
        with self.lock:
            return self.items.popleft()
        
class Worker(Thread):
    def __init__(self, func, in_queue, out_queue):
        super().__init__()
        self.func = func
        self.in_queue = in_queue
        self.out_queue = out_queue
        self.polled_count = 0
        self.work_done = 0

    def run(self):
        while True:
            self.polled_count += 1
            try:
                item = self.in_queue.get()
            except IndexError:
                sleep(0.01)
            else:
                result = self.func(item)
                self.out_queue.put(result)
                self.work_done += 1
                print('> work_done: %s' % self.work_done)

download_queue = MyQueue()
resize_queue = MyQueue()
upload_queue = MyQueue()
done_queue = MyQueue()
threads = [
    Worker(download, download_queue, resize_queue),
    Worker(resize, resize_queue, upload_queue),
    Worker(upload, upload_queue, done_queue),
]

count = 10

for thread in threads:
    thread.start()

for _ in range(count):
    download_queue.put(object())
    
while len(done_queue.items) < count:
        print('> wait...')
        sleep(1)
        
processed = len(done_queue.items)
polled = sum(t.polled_count for t in threads)
print('Processed', processed, 'items after polling', polled, 'times')

> wait...
download..
> wait...
> wait...
> wait...
> work_done: 1
download..
resize..
> wait...
> work_done: 1
upload...
> wait...
> work_done: 2
download..
resize..
> work_done: 1
> wait...
> work_done: 3
download..
> wait...
> work_done: 2
resize..
upload...
> wait...
> wait...
> work_done: 4
download..
> work_done: 2
> wait...
> work_done: 5
download..
> work_done: 3
resize..
upload...
> wait...
> wait...
> work_done: 6
download..
> wait...
> work_done: 4
resize..
> work_done: 3
upload...
> wait...
> work_done: 7
download..
> work_done: 5
resize..
> work_done: 4
upload...
> wait...
> wait...
> work_done: 8
download..
> work_done: 5
> wait...
> work_done: 6
resize..
upload...
> wait...
> work_done: 7
resize..
> wait...
> work_done: 9
download..
> work_done: 6
upload...
> wait...
> work_done: 10
> work_done: 8
resize..
> wait...
> work_done: 9
resize..
> work_done: 7
upload...
> wait...
> work_done: 10
> wait...
> wait...
> work_done: 8
upload...
> wait...
> wait...
> wait...
> work_d